In [5]:
from gensim.models import Word2Vec
import pandas as pd
import pickle
import time
import logging
import multiprocessing as mp
import os
logging.basicConfig(
	format='%(asctime)s : %(levelname)s : %(message)s',
	level=logging.INFO)
corpus_path = 'corpus/'
cores = mp.cpu_count()
name_corpus = ['attraction_tag.list',
               'hotel_tag.list',
               'restaurant_tag.list']

name_model = ['model/attraction_tag.model',
              'model/hotel_tag.model',
              'model/restaurant_tag.model']

params_tag = [{'size':300, 'window':99999, 'min_count':0,        # Attraction
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-2},
              {'size':300, 'window':99999, 'min_count':0,        # Hotel
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-4},
              {'size':300, 'window':99999, 'min_count':0,        # Restaurant
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-4}]

In [6]:
corpora = []
for name in name_corpus:
    try:
        os.stat(corpus_path+name)
        with open(corpus_path+name, 'rb') as f:
            corpora.append(pickle.load(f))
    except FileNotFoundError:
        # int to string
        with open(corpus_path+'kor_'+name, 'rb') as f:
            corpus = pickle.load(f)
        corpus = [[str(pid) for pid in line] for line in corpus]
        with open(corpus_path+'kor_'+name,'wb') as f:
            pickle.dump(corpus, f)
        with open(corpus_path+'eng_'+name, 'rb') as f:
            corpus = pickle.load(f)
        corpus = [[str(pid) for pid in line] for line in corpus]
        with open(corpus_path+'eng_'+name,'wb') as f:
            pickle.dump(corpus, f)
        # 한글&영문 corpus 병합
        with open(corpus_path+'kor_'+name, 'rb') as f:
            kor = pickle.load(f)
        with open(corpus_path+'eng_'+name, 'rb') as f:
            eng = pickle.load(f)
        merged = kor+eng
        with open(corpus_path+name, 'wb') as f:
            pickle.dump(merged, f)
        corpora.append(merged)

In [7]:
#test
corpus = corpora[0]
print('corpus 길이', len(corpus))
total = pd.DataFrame([len(sent) for sent in corpus]).sum()
length = 0
for line in corpus:
    length = max(length, len(line))
print('corpus내 최대 길이', length)
print('corpus내 모든 장소 합', total)

corpus 길이 65862
corpus내 최대 길이 33995
corpus내 모든 장소 합 0    2256328
dtype: int64


In [8]:
spent = []
#일부
for i in range(1,3):
#전체
#for i in range(len(corpora)):
    start = time.time()
    model = Word2Vec(corpora[i], **params_tag[i])
    spent.append('Elapsed time: '+str(time.time() - start)+' sec'+' ['+name_model[i]+']')
    model.wv.save(name_model[i])
print(spent)

2018-09-02 19:31:26,842 : INFO : collecting all words and their counts
2018-09-02 19:31:26,842 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-02 19:31:26,876 : INFO : PROGRESS: at sentence #10000, processed 261355 words, keeping 532 word types
2018-09-02 19:31:26,993 : INFO : PROGRESS: at sentence #20000, processed 1303452 words, keeping 972 word types
2018-09-02 19:31:27,002 : INFO : PROGRESS: at sentence #30000, processed 1356950 words, keeping 973 word types
2018-09-02 19:31:27,007 : INFO : PROGRESS: at sentence #40000, processed 1375840 words, keeping 975 word types
2018-09-02 19:31:27,010 : INFO : collected 980 word types from a corpus of 1383273 raw words and 45889 sentences
2018-09-02 19:31:27,010 : INFO : Loading a fresh vocabulary
2018-09-02 19:31:27,012 : INFO : effective_min_count=0 retains 980 unique words (100% of original 980, drops 0)
2018-09-02 19:31:27,013 : INFO : effective_min_count=0 leaves 1383273 word corpus (100% of original 13

2018-09-02 19:32:30,683 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-02 19:32:30,693 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-02 19:32:30,721 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 19:32:30,754 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 19:32:30,839 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 19:32:30,954 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 19:32:30,954 : INFO : EPOCH - 2 : training on 1383273 raw words (372110 effective words) took 32.1s, 11594 effective words/s
2018-09-02 19:32:32,110 : INFO : EPOCH 3 - PROGRESS: at 0.04% examples, 2134 words/s, in_qsize 16, out_qsize 0
2018-09-02 19:32:33,204 : INFO : EPOCH 3 - PROGRESS: at 0.45% examples, 9886 words/s, in_qsize 15, out_qsize 2
2018-09-02 19:32:34,335 : INFO : EPOCH 3 - PROGRESS: at 25.75% examples, 24037 words/s, in_qsize

2018-09-02 19:33:39,138 : INFO : EPOCH 5 - PROGRESS: at 25.78% examples, 19468 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:41,991 : INFO : EPOCH 5 - PROGRESS: at 25.80% examples, 12416 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:43,153 : INFO : EPOCH 5 - PROGRESS: at 25.80% examples, 10930 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:44,352 : INFO : EPOCH 5 - PROGRESS: at 25.81% examples, 10024 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:45,699 : INFO : EPOCH 5 - PROGRESS: at 25.81% examples, 9429 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:46,733 : INFO : EPOCH 5 - PROGRESS: at 25.83% examples, 8952 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:47,746 : INFO : EPOCH 5 - PROGRESS: at 25.86% examples, 8526 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:48,830 : INFO : EPOCH 5 - PROGRESS: at 25.88% examples, 8374 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:33:49,856 : INFO : EPOCH 5 - PROGRESS: at 25.96% examples, 8653 words/s, in_qsize 15, 

2018-09-02 19:34:56,186 : INFO : EPOCH 7 - PROGRESS: at 26.01% examples, 8011 words/s, in_qsize 16, out_qsize 0
2018-09-02 19:34:57,289 : INFO : EPOCH 7 - PROGRESS: at 26.07% examples, 7953 words/s, in_qsize 16, out_qsize 0
2018-09-02 19:34:58,338 : INFO : EPOCH 7 - PROGRESS: at 26.10% examples, 8008 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:34:59,391 : INFO : EPOCH 7 - PROGRESS: at 26.14% examples, 7908 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:35:00,638 : INFO : EPOCH 7 - PROGRESS: at 26.21% examples, 7679 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:35:01,812 : INFO : EPOCH 7 - PROGRESS: at 26.32% examples, 7699 words/s, in_qsize 16, out_qsize 0
2018-09-02 19:35:02,831 : INFO : EPOCH 7 - PROGRESS: at 26.40% examples, 7740 words/s, in_qsize 16, out_qsize 0
2018-09-02 19:35:03,922 : INFO : EPOCH 7 - PROGRESS: at 26.44% examples, 7605 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:35:04,939 : INFO : EPOCH 7 - PROGRESS: at 26.59% examples, 7840 words/s, in_qsize 15, out_

2018-09-02 19:36:09,967 : INFO : EPOCH 9 - PROGRESS: at 26.54% examples, 7660 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:36:11,101 : INFO : EPOCH 9 - PROGRESS: at 26.84% examples, 7746 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:36:12,150 : INFO : EPOCH 9 - PROGRESS: at 27.03% examples, 7891 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:36:13,296 : INFO : EPOCH 9 - PROGRESS: at 27.10% examples, 7759 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:36:14,376 : INFO : EPOCH 9 - PROGRESS: at 27.52% examples, 8062 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:36:15,440 : INFO : EPOCH 9 - PROGRESS: at 28.70% examples, 8656 words/s, in_qsize 15, out_qsize 0
2018-09-02 19:36:16,446 : INFO : EPOCH 9 - PROGRESS: at 34.63% examples, 9987 words/s, in_qsize 14, out_qsize 1
2018-09-02 19:36:16,548 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-02 19:36:16,569 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-02 19:36:16,577 : INFO : w

...

2018-09-03 18:43:29,342 : INFO : EPOCH 99 - PROGRESS: at 0.62% examples, 448 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:43:33,231 : INFO : EPOCH 99 - PROGRESS: at 0.66% examples, 458 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:43:36,702 : INFO : EPOCH 99 - PROGRESS: at 0.66% examples, 465 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:43:49,499 : INFO : EPOCH 99 - PROGRESS: at 0.67% examples, 460 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:44:08,777 : INFO : EPOCH 99 - PROGRESS: at 0.68% examples, 457 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:44:21,522 : INFO : EPOCH 99 - PROGRESS: at 0.72% examples, 462 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:44:29,958 : INFO : EPOCH 99 - PROGRESS: at 0.78% examples, 480 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:45:03,430 : INFO : EPOCH 99 - PROGRESS: at 0.81% examples, 469 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:45:04,729 : INFO : EPOCH 99 - PROGRESS: at 0.82% examples, 479 words/s, in_qsize 16, out_qsize 0
2

2018-09-03 18:51:30,120 : INFO : EPOCH 99 - PROGRESS: at 35.93% examples, 939 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:51:31,472 : INFO : EPOCH 99 - PROGRESS: at 35.96% examples, 942 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:51:32,535 : INFO : EPOCH 99 - PROGRESS: at 35.97% examples, 945 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:51:34,143 : INFO : EPOCH 99 - PROGRESS: at 35.99% examples, 947 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:51:35,523 : INFO : EPOCH 99 - PROGRESS: at 36.00% examples, 950 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:51:37,049 : INFO : EPOCH 99 - PROGRESS: at 36.00% examples, 955 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:51:40,752 : INFO : EPOCH 99 - PROGRESS: at 36.15% examples, 965 words/s, in_qsize 16, out_qsize 0
2018-09-03 18:51:42,462 : INFO : EPOCH 99 - PROGRESS: at 36.21% examples, 968 words/s, in_qsize 15, out_qsize 0
2018-09-03 18:51:44,286 : INFO : EPOCH 99 - PROGRESS: at 36.41% examples, 984 words/s, in_qsize 16, out_

2018-09-03 19:02:53,647 : INFO : EPOCH 100 - PROGRESS: at 0.78% examples, 487 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:03:17,649 : INFO : EPOCH 100 - PROGRESS: at 0.79% examples, 480 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:03:30,143 : INFO : EPOCH 100 - PROGRESS: at 0.82% examples, 484 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:03:35,357 : INFO : EPOCH 100 - PROGRESS: at 0.84% examples, 491 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:03:43,447 : INFO : EPOCH 100 - PROGRESS: at 0.90% examples, 497 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:03:49,871 : INFO : EPOCH 100 - PROGRESS: at 0.92% examples, 504 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:03:53,066 : INFO : EPOCH 100 - PROGRESS: at 0.97% examples, 514 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:03:56,984 : INFO : EPOCH 100 - PROGRESS: at 1.03% examples, 522 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:04:23,314 : INFO : EPOCH 100 - PROGRESS: at 1.07% examples, 513 words/s, in_qsize 15, out_

2018-09-03 19:09:45,836 : INFO : EPOCH 100 - PROGRESS: at 35.94% examples, 954 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:09:47,590 : INFO : EPOCH 100 - PROGRESS: at 35.96% examples, 956 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:09:49,343 : INFO : EPOCH 100 - PROGRESS: at 35.99% examples, 969 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:09:53,355 : INFO : EPOCH 100 - PROGRESS: at 36.05% examples, 974 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:09:56,216 : INFO : EPOCH 100 - PROGRESS: at 36.16% examples, 981 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:09:57,391 : INFO : EPOCH 100 - PROGRESS: at 36.32% examples, 994 words/s, in_qsize 16, out_qsize 0
2018-09-03 19:09:58,661 : INFO : EPOCH 100 - PROGRESS: at 36.36% examples, 997 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:10:03,472 : INFO : EPOCH 100 - PROGRESS: at 36.43% examples, 1001 words/s, in_qsize 15, out_qsize 0
2018-09-03 19:10:05,067 : INFO : EPOCH 100 - PROGRESS: at 36.49% examples, 1004 words/s, in_qsi

FileNotFoundError: [Errno 2] No such file or directory: 'model/restaurant_tag.model'

In [33]:
with open('douchebag', 'wb') as f:
    pickle.dump(model.wv, f)

FileNotFoundError: [Errno 2] No such file or directory: 'douchebag'

In [28]:
pwd

'F:\\소공전프로젝트\\sofcon\\recomm'

In [30]:
ls model

 F 드라이브의 볼륨: External
 볼륨 일련 번호: 7284-6386

 F:\소공전프로젝트\sofcon\recomm\model 디렉터리

2018-09-03  오후 07:37    <DIR>          .
2018-09-03  오후 07:37    <DIR>          ..
2018-08-30  오후 06:03    <DIR>          .ignore
2018-09-02  오전 11:41           984,661 attraction_tag_e2.model
2018-09-01  오후 03:18           984,939 attraction_tag_e3.model
2018-09-01  오전 11:10           986,382 attraction_tag_e4.model
2018-08-30  오후 06:05           981,339 attraction_tag_e5.model
2018-08-20  오후 07:13         1,057,984 attraction_user.model
2018-09-02  오후 08:31         1,819,680 hotel_tag.model
2018-08-30  오후 06:06         1,818,867 hotel_tag_e5.model
2018-08-20  오후 07:13         1,950,191 hotel_user.model
2018-09-03  오후 07:36                 0 restaurant_tag_e4.model
2018-09-02  오후 04:12        41,791,000 restaurant_tag_e5.model
2018-08-20  오후 07:13        40,800,323 restaurant_user.model
              11개 파일          93,175,366 바이트
               3개 디렉터리  223,044,087,808 바이트 남음
